In [8]:
%matplotlib inline
import pandas as pd
from scipy.stats import pearsonr, spearmanr
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np

lang2mono = {'EN': 'bert-base-uncased',
             'ID': 'indobert-base-uncased',
             'FR': 'camembert-base',
             'TR': 'bert-base-turkish-uncased',
             'ZH': 'bert-base-chinese',
             'RU': 'rubert-base-cased',
             'DE': 'bert-base-german-dbmdz-uncased',
             'ES': 'bert-base-spanish-wwm-uncased'}

LANGS = ['EN', 'ID', 'FR', 'TR', 'ZH', 'RU', 'DE', 'ES']

In [9]:
def read_human_annotation(lang, types='focus'):
    if types=='focus':
        path_human = f'mturk/annotation_result/{lang}/human_focus_final.csv'
    else:
        assert types=='coverage'
        path_human = f'mturk/annotation_result/{lang}/human_coverage_final.csv'
    human = pd.read_csv(path_human)
    return human

def align(df, human_score, model):
    human = []
    machine = []
    #print(df)
    for idx, row in df.iterrows():
        human.append(human_score[(human_score['model'] == model) & (human_score['id']==row['fnames'])]['score'].values[0])
        machine.append(row['moverscore'])
    return human, machine

def read(lang, human_score, pretrained):
    path_BERT = f'mover_score/{lang}--BERT--{pretrained}.csv'
    path_PG = f'mover_score/{lang}--PG--{pretrained}.csv'
        
    humans = []; machines = []

    df_BERT = pd.read_csv(path_BERT)
    human, machine = align(df_BERT, human_score, 'BERT')
    humans += human
    machines += machine
    
    df_PG = pd.read_csv(path_PG)
    human, machine = align(df_PG, human_score, 'PG')
    humans += human
    machines += machine
    
    #print(humans)
    #print(machines)
    #return spearmanr(humans, machines)[0], len(humans)
    return pearsonr(humans, machines)[0], len(humans)

In [10]:
LANGS = ['EN', 'ID', 'FR', 'TR', 'ZH', 'RU', 'DE', 'ES']

# Focus
print('Focus')
cors = {}
for lang in LANGS:
    human_score = read_human_annotation(lang, 'focus')

    for pretrained in [ lang2mono[lang], 'bert-base-multilingual-cased', 'bert-base-multilingual-uncased', \
                      'xlm-roberta-base', 'xlm-roberta-large']:    
        cor, num = read(lang, human_score, pretrained)
        print(lang, num, pretrained, cor)    
    print()

Focus
EN 270 bert-base-uncased 0.5802248969741917
EN 270 bert-base-multilingual-cased 0.5387837105939391
EN 270 bert-base-multilingual-uncased 0.5892230639678848
EN 270 xlm-roberta-base 0.5259703002291909
EN 270 xlm-roberta-large 0.5051756362902975

ID 270 indobert-base-uncased 0.6502816035615807
ID 270 bert-base-multilingual-cased 0.6753557391772896
ID 270 bert-base-multilingual-uncased 0.6894454416099718
ID 270 xlm-roberta-base 0.6260939963618947
ID 270 xlm-roberta-large 0.5830980758325219

FR 270 camembert-base 0.7119766159090477
FR 270 bert-base-multilingual-cased 0.765452543428397
FR 270 bert-base-multilingual-uncased 0.7748925546498105
FR 270 xlm-roberta-base 0.6861086177502986
FR 270 xlm-roberta-large 0.6698015012533034

TR 270 bert-base-turkish-uncased 0.8154763953267194
TR 270 bert-base-multilingual-cased 0.7908396207454944
TR 270 bert-base-multilingual-uncased 0.8108987780857521
TR 270 xlm-roberta-base 0.8038763730118588
TR 270 xlm-roberta-large 0.7892349565370103

ZH 270 ber

In [11]:
# Coverage
print('Coverage')

cors = {}
for lang in LANGS:
    human_score = read_human_annotation(lang, 'coverage')
    for pretrained in [ lang2mono[lang], 'bert-base-multilingual-cased', 'bert-base-multilingual-uncased', \
                      'xlm-roberta-base', 'xlm-roberta-large']: 
        cor, num = read(lang, human_score, pretrained)
        print(lang, num, pretrained, cor)
    print()

Coverage
EN 270 bert-base-uncased 0.5942562985333694
EN 270 bert-base-multilingual-cased 0.5244621002608667
EN 270 bert-base-multilingual-uncased 0.5872798684003329
EN 270 xlm-roberta-base 0.5766830654535068
EN 270 xlm-roberta-large 0.5497836265813958

ID 270 indobert-base-uncased 0.6240390018610549
ID 270 bert-base-multilingual-cased 0.6871406004218968
ID 270 bert-base-multilingual-uncased 0.6890553084287263
ID 270 xlm-roberta-base 0.6237625780863463
ID 270 xlm-roberta-large 0.6149855769356989

FR 270 camembert-base 0.6713135396295669
FR 270 bert-base-multilingual-cased 0.7218578258116559
FR 270 bert-base-multilingual-uncased 0.7484409659661108
FR 270 xlm-roberta-base 0.6246436497490657
FR 270 xlm-roberta-large 0.5867571249185946

TR 270 bert-base-turkish-uncased 0.7828815692563675
TR 270 bert-base-multilingual-cased 0.753117180414588
TR 270 bert-base-multilingual-uncased 0.7688175164975413
TR 270 xlm-roberta-base 0.7420722179927229
TR 270 xlm-roberta-large 0.7196229657060806

ZH 270 